# App Test Drive
* In this notebook we'll sequentially go thru each steps of the app, before we convert into a fully fledged web app. 

## Import Libraries

In [5]:
import os
import json

import PIL
from google import genai
from PIL import Image
from io import BytesIO
from IPython.display import display

from pathlib import Path
from datetime import datetime

## Setup Constants

In [35]:
image_dir = Path("..","images")
ai_image_dir = Path(image_dir,"ai_images")
data_dir = Path("..", "data")

IMAGE_MODEL_ID = "gemini-2.5-flash-image-preview"
TEXT_MODEL_ID = "gemini-2.5-flash-lite"

## Client Setup

In [ ]:
# read api key
API_KEY = os.environ.get("GEMINI_API_KEY")


# configure the client with your API key
client = genai.Client(api_key=API_KEY)

## Utils

In [4]:
def save_and_display_response(response, file_name = f"image_{datetime.now().timestamp()}.png"):
    image_parts = [
        part.inline_data.data
        for part in response.candidates[0].content.parts
        if part.inline_data
    ]
    
    if image_parts:
        image = Image.open(BytesIO(image_parts[0]))
        image.save(f"{ai_image_dir}/{file_name}")
        display(image)

## Step 1 - Setup Screen

* In this screen we'll give users few choices of story theme and based on their selection we'll create the story and character for them. 
* We'll use GenAI to generate this story and will try and use it to create prompts for image and text. 

### Prompt

In [29]:
setup_screen_prompt_template = """
You are a creative and expert Dungeon Master AI. Your task is to design a short, branching adventure story based on a user-provided theme and a desired number of steps for a successful playthrough.

The theme is: "{theme}"
The successful path length must be exactly: {step_count} steps.

Your output MUST be a single, valid JSON object. Do not include any text, notes, or markdown formatting (like ```json) before or after the JSON block.

The JSON object must conform to the following rules and schema:

1.  **prologue**: A compelling background story (2-3 sentences) explaining the hero's motivation and setting the scene.
2.  **story_tree**: This must be a LIST of story node objects. The first object in the list is the starting point of the adventure and MUST have its id field set to "start".
3.  **scene_description**: A visually rich and descriptive text prompt (2-3 sentences) for an AI image generator.
4.  **narration**: Text for an AI voice to narrate the scene (2-3 sentences).
5.  **choices**: An array of 1 or 2 choice objects. Ending nodes must have an empty `[]` choices array.
6.  **is_ending**: A boolean (`true` or `false`). This MUST be `true` for any node that represents an end to the game.
7.  **outcome**: A string, either `"success"` or `"failure"`. This key MUST be present if and only if `is_ending` is `true`.
8.  **Failure Paths**: You MUST include at least one path that leads to a `failure` outcome. Failure paths can be shorter than the successful path.

Here is the exact schema for each node within the `story_tree`:
{{
  "id": "string",
  "scene_description": "string",
  "narration": "string",
  "choices": [
    {{
      "text": "string",
      "next_id": "string"
    }}
  ],
  "is_ending": "boolean",
  "outcome": "string (optional: 'success' or 'failure')"
}}

Now, generate the complete JSON for my adventure.
"""

### Reponse Model

In [30]:
from pydantic import BaseModel
from typing import List, Optional, Literal

# No changes are needed for the Choice class.
class Choice(BaseModel):
    """
    Represents a single choice a player can make.
    """
    text: str
    next_id: str


class StoryNode(BaseModel):
    """
    Represents a single step in the story.
    We've added the 'id' field here to uniquely identify each node.
    """
    id: str  # <-- The new field you suggested!
    scene_description: str
    narration: str
    choices: List[Choice]
    is_ending: bool
    outcome: Optional[Literal["success", "failure"]] = None


class Story(BaseModel):
    """
    Represents the entire adventure.
    The 'story_tree' is now a List of StoryNode objects instead of a Dictionary.
    This resolves the `additionalProperties` error with the Gemini API.
    """
    prologue: str
    story_tree: List[StoryNode]

### Call

In [ ]:
# --- Define Your Game Parameters ---
theme = "A rescue mission in a haunted, abandoned space station"
step_count = 3
final_prompt = setup_screen_prompt_template.format(theme=theme, step_count=step_count)

## Commenting out this part to avoid multiple API calls if we restart the notebook

# --- Make the API Call ---
# response = client.models.generate_content(
#     model=TEXT_MODEL_ID,
#     contents=final_prompt,
#     config={
#         "response_mime_type": "application/json",
#         "response_schema":Story
#     }
# )
# story_json_text = response.text

# # --- Parse and Print the Story ---
# story_data = json.loads(story_json_text)

# # Pretty print the result to inspect it
# import pprint
# pprint.pprint(story_data)

{'prologue': 'A distress signal, faint but desperate, has been traced to the '
             "derelict starship 'Odyssey,' lost years ago in the treacherous "
             'Nebula X-7. Your mission: board the station, locate the '
             'survivor, and bring them back alive. But rumors swirl of the '
             "'Odyssey' being haunted, a tomb for its entire crew.",
 'story_tree': [{'choices': [{'next_id': 'corridor_approach',
                              'text': 'Investigate the tapping sound.'},
                             {'next_id': 'bridge_approach',
                              'text': 'Head towards the bridge for station '
                                      'logs.'}],
                 'id': 'start',
                 'is_ending': False,
                 'narration': 'You cautiously step onto the Odyssey, the heavy '
                              'airlock door sealing behind you with a clang. '
                              'The silence is unnerving. A faint, rhythmic

In [ ]:
## read story data from saved json


{'prologue': "A distress signal, faint but desperate, has been traced to the derelict starship 'Odyssey,' lost years ago in the treacherous Nebula X-7. Your mission: board the station, locate the survivor, and bring them back alive. But rumors swirl of the 'Odyssey' being haunted, a tomb for its entire crew.",
 'story_tree': [{'id': 'start',
   'scene_description': 'The airlock hisses open, revealing a dimly lit, dust-covered corridor of an abandoned space station. Emergency lights flicker erratically, casting long, dancing shadows. The silence is oppressive, broken only by the distant hum of failing machinery.',
   'narration': 'You cautiously step onto the Odyssey, the heavy airlock door sealing behind you with a clang. The silence is unnerving. A faint, rhythmic tapping echoes from deeper within the ship. Your objective is clear: find any survivors.',
   'choices': [{'text': 'Investigate the tapping sound.',
     'next_id': 'corridor_approach'},
    {'text': 'Head towards the bridge

In [36]:
with open(Path(data_dir,"adventure.json"), "r") as file:
    story_data = json.load(file)
    
story_data

{'prologue': "A distress signal, faint but desperate, has been traced to the derelict starship 'Odyssey,' lost years ago in the treacherous Nebula X-7. Your mission: board the station, locate the survivor, and bring them back alive. But rumors swirl of the 'Odyssey' being haunted, a tomb for its entire crew.",
 'story_tree': [{'id': 'start',
   'scene_description': 'The airlock hisses open, revealing a dimly lit, dust-covered corridor of an abandoned space station. Emergency lights flicker erratically, casting long, dancing shadows. The silence is oppressive, broken only by the distant hum of failing machinery.',
   'narration': 'You cautiously step onto the Odyssey, the heavy airlock door sealing behind you with a clang. The silence is unnerving. A faint, rhythmic tapping echoes from deeper within the ship. Your objective is clear: find any survivors.',
   'choices': [{'text': 'Investigate the tapping sound.',
     'next_id': 'corridor_approach'},
    {'text': 'Head towards the bridge